<a href="https://colab.research.google.com/github/pnpsuM/Udacity/blob/main/FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U tensorflow_datasets

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import tensorflow_datasets as tfds

import math
import numpy as np
import matplotlib.pyplot as plt

from tqdm import auto, tqdm
tqdm = auto.tqdm

print(tf.__version__)

In [ ]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
train_ds, test_ds = dataset['train'], dataset['test']

In [ ]:
class_names = ['T/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker',
               'Bag', 'Ankle boot']
num_train_examples = metadata.splits['train'].num_examples

In [ ]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_ds = train_ds.map(normalize)
test_ds = test_ds.map(normalize)

In [ ]:
for image, label in test_ds.take(1):
  break
image = image.numpy().reshape((28, 28))

plt.figure()
plt.imshow(image, cmap = plt.cm.binary)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
for i, (image, label) in enumerate(test_ds.take(25)):
  if i == 0:
    print(image.shape)
  image = image.numpy().reshape((28,28))
  plt.subplot(5, 5, i + 1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(image, cmap = plt.cm.binary)
  plt.xlabel(class_names[label])
plt.show()

In [ ]:
from keras import Sequential
from keras.layers import Dense, Flatten

model = Sequential([
    Flatten(input_shape = (28,28,1)),
    Dense(128, tf.nn.relu),
    Dense(10, tf.nn.softmax)
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['acc'])
model.summary()

In [ ]:
BATCH_SIZE = 32
train_ds = train_ds.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
history = model.fit(train_ds, epochs = 5, steps_per_epoch=tf.math.ceil(num_train_examples/BATCH_SIZE))

In [ ]:
num_test_examples = metadata.splits['test'].num_examples
test_loss, test_acc = model.evaluate(test_ds, steps = tf.math.ceil(num_test_examples / BATCH_SIZE))
print(test_acc)

In [ ]:
for test_images, test_labels in test_ds.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [ ]:
print(predictions[0])
print(np.argmax(predictions[0]))
print(test_labels[0], class_names[test_labels[0]])

In [ ]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel(
      f"{class_names[predicted_label]} {100*np.max(predictions_array):2.0f}% ({class_names[true_label]})".format(color=color))

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 30
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)